<a href="https://colab.research.google.com/github/ViniciusVitto/MachineLearning/blob/main/MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de Doenças Cardíacas com Machine Learning

Neste projeto, vamos utilizar **técnicas de machine learning** para prever a presença de doenças cardíacas em pacientes. Para isso, utilizaremos o conjunto de dados de doenças cardíacas da UCI.

Nosso objetivo é construir diferentes modelos de classificação, otimizar seus hiperparâmetros e compará-los para encontrar o melhor desempenho.

## Objetivos:
1. Treinar modelos básicos de machine learning (KNN, Árvore de Decisão, Naive Bayes, SVM).
2. Otimizar os hiperparâmetros dos modelos KNN, Árvore de Decisão e SVM.
3. Comparar os desempenhos antes e depois da otimização.

In [1]:
# Importar bibliotecas necessárias
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
import joblib

### Carregamento e Limpeza dos Dados

Vamos carregar o conjunto de dados de doenças cardíacas da UCI e realizar uma limpeza básica, removendo valores faltantes.


In [2]:
# Carregar o dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
df = pd.read_csv(url, names=columns)

# Verificar se há valores faltantes
df.replace('?', pd.NA, inplace=True)
df.dropna(inplace=True)

df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


### Divisão dos Dados e Pré-processamento

Agora vamos dividir os dados em treino e teste, e normalizá-los para que possam ser usados nos modelos de machine learning.


In [3]:
# Dividir entre features (X) e target (y)
X = df.drop('target', axis=1)
y = df['target']

# Separar dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Treinamento dos Modelos Básicos

Vamos treinar os modelos básicos de KNN, Árvore de Decisão, Naive Bayes e SVM, e avaliar o desempenho de cada um com base na acurácia e F1-Score.


In [5]:
# Função para calcular e exibir as métricas
def evaluate_model(model, X_test, y_test, name):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"\nDesempenho do {name}:")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"F1-Score: {f1:.4f}")
    return accuracy, f1

# Função para treinar e salvar o modelo
def train(model):
    model.fit(X_train_scaled, y_train)
    return model

# Treinar e avaliar KNN
knn = KNeighborsClassifier()
train(knn)
evaluate_model(knn, X_test_scaled, y_test, 'KNN')

# Treinar e avaliar Árvore de Decisão
tree = DecisionTreeClassifier()
train(tree)
evaluate_model(tree, X_test_scaled, y_test, 'Árvore de Decisão')

# Treinar e avaliar Naive Bayes
nb = GaussianNB()
train(nb)
evaluate_model(nb, X_test_scaled, y_test, 'Naive Bayes')

# Treinar e avaliar SVM
param_grid_svm = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}
grid_svm = GridSearchCV(SVC(), param_grid_svm, refit=True, cv=5)
grid_svm.fit(X_train_scaled, y_train)
evaluate_model(grid_svm, X_test_scaled, y_test, 'SVM')


Desempenho do KNN:
Acurácia: 0.6333
F1-Score: 0.5982

Desempenho do Árvore de Decisão:
Acurácia: 0.5667
F1-Score: 0.5927

Desempenho do Naive Bayes:
Acurácia: 0.5333
F1-Score: 0.5602

Desempenho do SVM:
Acurácia: 0.6000
F1-Score: 0.5564


(0.6, 0.5564102564102564)

### Otimização dos Hiperparâmetros

Vamos otimizar os hiperparâmetros dos modelos KNN, Árvore de Decisão e SVM utilizando `GridSearchCV`.


In [6]:
# Função para otimizar e salvar o modelo otimizado
def optimize(model, param_grid, name):
    grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid.fit(X_train_scaled, y_train)
    best_model = grid.best_estimator_
    print(f"\nMelhor hiperparâmetro para {name}: {grid.best_params_}")
    return best_model

# Otimização de KNN
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
best_knn = optimize(KNeighborsClassifier(), param_grid_knn, 'knn')
evaluate_model(best_knn, X_test_scaled, y_test, 'KNN Otimizado')

# Otimização de Árvore de Decisão
param_grid_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
best_tree = optimize(DecisionTreeClassifier(), param_grid_tree, 'tree')
evaluate_model(best_tree, X_test_scaled, y_test, 'Árvore de Decisão Otimizada')

# Otimização de SVM já foi realizada anteriormente
best_svm = grid_svm.best_estimator_
evaluate_model(best_svm, X_test_scaled, y_test, 'SVM Otimizado')



Melhor hiperparâmetro para knn: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}

Desempenho do KNN Otimizado:
Acurácia: 0.6500
F1-Score: 0.6126

Melhor hiperparâmetro para tree: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2}

Desempenho do Árvore de Decisão Otimizada:
Acurácia: 0.5833
F1-Score: 0.5843

Desempenho do SVM Otimizado:
Acurácia: 0.6000
F1-Score: 0.5564


(0.6, 0.5564102564102564)

### Conclusão

Após a otimização, podemos ver que os modelos melhoraram seus desempenhos em termos de acurácia e F1-Score. O **KNN Otimizado** apresentou os melhores resultados entre os modelos treinados.
